In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [3]:
with open('datasets/sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

### let’s tokenize the text to create a sequence of words:

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [25]:
total_words

8200

### let’s create input-output pairs by splitting the text into sequences of tokens and forming n-grams from the sequences:

In [5]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [16]:
len(input_sequences)

96314

### let’s pad the input sequences to have equal length:

In [7]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [8]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [9]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [10]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)


Epoch 1/100


3010/3010 [==============================] - 92s 30ms/step - loss: 6.2215 - accuracy: 0.0782
Epoch 2/100
3010/3010 [==============================] - 96s 32ms/step - loss: 5.4963 - accuracy: 0.1256
Epoch 3/100
3010/3010 [==============================] - 75s 25ms/step - loss: 5.1150 - accuracy: 0.1472
Epoch 4/100
3010/3010 [==============================] - 73s 24ms/step - loss: 4.7849 - accuracy: 0.1657
Epoch 5/100
3010/3010 [==============================] - 77s 26ms/step - loss: 4.4810 - accuracy: 0.1839
Epoch 6/100
3010/3010 [==============================] - 73s 24ms/step - loss: 4.1966 - accuracy: 0.2038
Epoch 7/100
3010/3010 [==============================] - 78s 26ms/step - loss: 3.9281 - accuracy: 0.2278
Epoch 8/100
3010/3010 [==============================] - 82s 27ms/step - loss: 3.6713 - accuracy: 0.2577
Epoch 9/100
3010/3010 [==============================] - 80s 26ms/step - loss: 3.4290 - accuracy: 0.2901
Epoch 10/100
3010/3010 [============================

In [58]:
seed_text = "I will leave if they"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 33ms/step
I will leave if they would have the
